# 환경설정

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [5]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "./chromedriver"
# driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

# 음식점 별 상세보기를 리스트로 가져오기

In [2]:
import re
import time

# brew 로 설치된 chromedriver의 path (Mac)
path = '/usr/local/bin/chromedriver'

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
excutable_path = 'chromedriver.exe'

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://map.kakao.com/"

# 크롬 드라이버를 사용합니다 (맥은 첫 줄, 윈도우는 두번째 줄 실행)
# driver = webdriver.Chrome(path)
driver = webdriver.Chrome(executable_path=excutable_path)

# 카카오 지도에 접속합니다
driver.get(source_url)

# 검색창에 검색어를 입력합니다
searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
searchbox.send_keys("인사동 맛집")

# 검색버튼을 눌러서 결과를 가져옵니다
searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
driver.execute_script("arguments[0].click();", searchbutton)

# 검색 결과를 가져올 시간을 기다립니다
time.sleep(2)

# 검색 결과의 페이지 소스를 가져옵니다
html = driver.page_source


driver.find_element(By.XPATH, '/html/body/div[10]/div/div/div/span').click()
time.sleep(2)
#더보기 클릭 
driver.find_element(By.XPATH, "//*[@id='info.search.place.more']").click()
time.sleep(2)
# 1~ 5페이지 링크 얻기
page_urls = []
for i in range(1,6):
    page = driver.find_element(By.ID, 'info.search.page.no' + str(i))
    page.click()
    time.sleep(2)
    urls = driver.find_elements(By.LINK_TEXT, "상세보기")
    for j in range(len(urls)):
        url = urls[j].get_attribute('href')
        print(url)
        page_urls.append(url)
        


C:\Users\max_s\AppData\Local\Temp/ipykernel_57200/2396452402.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=excutable_path)
C:\Users\max_s\AppData\Local\Temp/ipykernel_57200/2396452402.py:21: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
C:\Users\max_s\AppData\Local\Temp/ipykernel_57200/2396452402.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")


NameError: name 'By' is not defined

## 상세보기 링크 개수 확인

In [ ]:
len(page_urls)

## 링크를 하나씩 열어 음식점별 정보를 DF형태로 저장

In [ ]:
columns = ['name','addresse','cat1','cat2','review_num','score', 'review','reviews_dates']
df = pd.DataFrame(columns=columns)

# driver = webdriver.Chrome(path)  # for Mac
driver = webdriver.Chrome(executable_path=excutable_path)  # for Windows
for page_url in page_urls:

    # 상세보기 페이지에 접속합니다
    driver.get(page_url)
    time.sleep(5)
    #사업장명
    name =  driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text
    # 사업장 주소
    address = driver.find_element(By.CLASS_NAME, "txt_address").text
    # 음식 종류
    cat1 =  driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/span[1]').text
    # 메뉴
    cat2 = []
    menus = driver.find_elements(By.CLASS_NAME, "info_menu")
    for menu in menus:
        cat2.append(menu.text)
    if driver.find_elements(By.CLASS_NAME, "box_grade_off"): #후기를 제공하지 않는 맛집 넘기기
        continue
    else:
        #리뷰수
        review_num = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[2]/span').text
        # 첫 페이지 리뷰를 크롤링합니다
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

        # 별점을 가져옵니다.
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})

        # 리뷰를 가져옵니다.
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
        
        # 리뷰를 쓴 날짜를 가져옵니다.
        reviews_dates = contents_div.find_all(name="span", attrs={"class":"time_write"})

        for rate, review,reviews_date in zip(rates, reviews,reviews_dates):
            row = [name,address,cat1,cat2,review_num,rate.text[0], review.find(name="span").text,reviews_date.text]
            series = pd.Series(row, index=df.columns)
            df = df.append(series, ignore_index=True)


        # 2-5페이지의 리뷰를 크롤링합니다
        for button_num in range(2, 6):
            # 오류가 나는 경우(리뷰 페이지가 없는 경우), 수행하지 않습니다.
            try:
                another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
                another_reviews.click()
                time.sleep(2)

                # 페이지 리뷰를 크롤링합니다
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

                # 별점을 가져옵니다.
                rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
                # 리뷰를 가져옵니다.
                reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
                
                # 리뷰를 쓴 날짜를 가져옵니다.
                reviews_dates = contents_div.find_all(name="span", attrs={"class":"time_write"})

                for rate, review,reviews_date in zip(rates, reviews,reviews_dates):
                    row = [name,address,cat1,cat2,review_num,rate.text[0], review.find(name="span").text,reviews_date.text]
                    series = pd.Series(row, index=df.columns)
                    df = df.append(series, ignore_index=True)
            except:
                break    
driver.close()

# 위 과정을 한번에 처리하고 여러 키워드를 반복해서 돌리기
- 키워드만 넣어주고 한번에 처리 가능하다

### 호선별 중복 역 색출
- a 가 1호선으로 쭉 나간다
- 먼저 문자열을 split로 리스트한 뒤 set으로 타입변경한다.
- 교집합이 중복된 역이다
- 차집합으로 앞선 호선에서 중복된 역 외의 정보만 반환한다.

In [ ]:
a = '남영, 용산, 노량진, 대방, 신길, 영등포, 신도림, 구로, 가산디지털단지, 독산, 금천구청 , 석수, 관악, 안양, 명학, 금정, 군포, 당정, 의왕, 성균관대, 화서, 수원, 세류, 병점 , 세마, 오산대, 오산, 진위, 송탄, 서정리, 지제, 평택, 성환, 직산, 두정, 천안, 금천구청 , 광명, 봉명, 쌍용, 아산, 배방, 온양온천, 신창, 병점, 서동탄, 구로, 구일, 개봉, 오류동, 온수, 역곡, 소사, 부천, 중동, 송내, 부개, 부평, 백운, 동암, 간석, 주안, 도화, 제물포, 도원, 동인천, 인천'
b = '성수, 뚝섬, 한양대, 왕십리, 상왕십리, 신당, 동대문역사문화공원, 을지로4가, 을지로3가, 을지로입구, 시청, 충정로, 아현, 이대, 신촌, 홍대입구, 합정, 강변, 구의, 건대입구, 당산, 영등포구청, 문래, 신도림, 대림, 구로디지털단지, 신대방, 신림, 봉천, 서울대입구, 낙성대, 사당, 방배, 서초, 교대, 강남, 역삼, 선릉, 삼성, 종합운동장, 잠실새내, 잠실나루'
c = '지축, 구파발, 연신내, 불광, 녹번, 홍제, 무악제, 독립문, 경복궁, 안국, 종로3가, 을지로3가, 충무로, 동대입구, 약수, 금호, 옥수, 압구정, 신사, 잠원, 고속터미널, 교대, 남부터미널, 양재 , 매봉, 도곡, 대치, 학여울, 대청, 일원, 수서, 가락시장, 경찰병원, 오금'
d = '당고개, 상계, 노원, 창동, 쌍문, 수유, 미아, 미아사거리, 길음, 성신여대입구, 한성대입구, 혜화, 동대문, 동대문역사문화공원 , 충무로, 명동, 회현, 서울역, 숙대입구, 삼각지, 신용산, 이촌, 동작, 총신대입구, 사당, 남태령'
e = '방화, 개화산, 김포공항, 송정, 마곡, 발산, 우장선, 화곡, 까치산, 신정, 목동, 오목교, 양평, 영등포구청, 영등포시장, 신길, 여의도, 여의나루, 마포, 공덕, 애오개, 충정로, 서대문, 광화문, 종로3가, 을지로4가, 동대문역사문화공원, 청구, 신금호, 행당, 왕십리, 마장, 답십리, 장한평, 군자, 아차산, 광나루, 천호, 강동, 길동, 굽은다리, 명일, 고덕, 상일동'
f = '응암, 역촌, 불광, 독바위, 연신내, 구산, 새절, 증산, 디지털미디어시티, 월드컵경기장, 마포구청, 망원, 합정, 상수, 광흥창, 대흥, 공덕, 효창공원앞, 삼각지, 녹사평, 이태원, 한강진, 버티고개, 약수, 청구, 신당, 동묘앞, 창신, 보문, 안암, 고려대, 월곡, 상월곡, 돌곶이, 석계, 태릉입구, 화랑대, 봉화산'
g = '장암, 도봉산, 수락산, 마들, 노원, 중계, 하계, 공릉, 태릉입구, 먹골, 중화, 상봉, 면목, 사가정, 용마산, 중곡, 군자, 어린이대공원, 건대입구, 뚝섬유원지, 청담, 강남구청, 학동, 논현, 반포, 고속터미널, 내방, 이수, 남성, 숭실대입구, 상도, 장승배기, 신대방삼거리, 보라매, 신풍, 대림, 남구로, 가산디지털단지, 철산, 광명사거리, 천왕, 온수, 까치울, 부천종합운동장, 춘의, 신중동, 부천시청, 상동, 삼산체육관, 굴포천, 부평구청'


In [ ]:
[a+b+c+d]

In [ ]:
lst1 = a+b+c+d # 앞선 호선 리스트화
tmp1 = lst1.split(', ')
set1 = set(tmp1)

In [ ]:
lst2 = e+f+g
tmp2 = lst2.split(', ')
set2 = set(tmp2)

In [ ]:
same = set1 & set2 #교집합이 중복된 역이다
same

In [ ]:
tmp3 = {'답십리','까치산','김포공항','길동','굽은다리','군자','광화문','공덕','고덕','개화산','강동'}

In [ ]:
set2 - same #차집합으로 앞선 호선에서 중복된 역 외의 정보만 반환

In [ ]:
set2 -= tmp3
list(set2)

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

import re
import time

# brew 로 설치된 chromedriver의 path (Mac)
path = '/usr/local/bin/chromedriver'

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
excutable_path = 'chromedriver.exe'

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://map.kakao.com/"

# 크롬 드라이버를 사용합니다 (맥은 첫 줄, 윈도우는 두번째 줄 실행)
# driver = webdriver.Chrome(path)

# 카카오 지도에 접속합니다

regions = ['신당',
 '석계',
 '우장선',
 '한강진',
 '중곡',
 '부천종합운동장',
 '보라매',
 '종로3가',
 '봉화산장암',
 '청구',
 '태릉입구',
 '마들',
 '새절',
 '마포',
 '약수',
 '영등포구청',
 '어린이대공원',
 '화랑대',
 '합정',
 '이태원',
 '신길',
 '애오개',
 '방화',
 '여의도',
 '장한평',
 '춘의',
 '충정로',
 '신대방삼거리',
 '양평',
 '역촌',
 '화곡',
 '창신',
 '철산',
 '송정',
 '동대문역사문화공원',
 '이수',
 '마포구청',
 '부평구청',
 '삼산체육관',
 '신중동',
 '남구로',
 '대림',
 '녹사평',
 '명일',
 '버티고개',
 '동묘앞',
 '뚝섬유원지',
 '효창공원앞',
 '상수',
 '대흥',
 '청담',
 '독바위',
 '월곡',
 '도봉산',
 '면목',
 '영등포시장',
 '구산',
 '반포',
 '내방',
 '장승배기',
 '월드컵경기장',
 '여의나루',
 '신풍',
 '상도',
 '상봉',
 '고려대',
 '남성',
 '을지로4가',
 '불광',
 '하계',
 '공릉',
 '부천시청',
 '강남구청',
 '행당',
 '건대입구',
 '신금호',
 '신정',
 '증산',
 '광나루',
 '디지털미디어시티',
 '노원',
 '논현',
 '용마산',
 '삼각지',
 '광흥창',
 '상동',
 '먹골',
 '망원',
 '가산디지털단지',
 '굴포천'] #불러올 키워드

for region in regions:
    driver = webdriver.Chrome(executable_path=excutable_path)
    driver.get(source_url) # 받아온 링크를 차례로 열기
    print(region)
    # 검색창에 검색어를 입력합니다
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
    searchbox.send_keys("{0}역 맛집".format(region))

    # 검색버튼을 눌러서 결과를 가져옵니다
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
    driver.execute_script("arguments[0].click();", searchbutton)

    # 검색 결과를 가져올 시간을 기다립니다
    time.sleep(2)

    # 검색 결과의 페이지 소스를 가져옵니다
    html = driver.page_source


    driver.find_element(By.XPATH, '/html/body/div[10]/div/div/div/span').click() #소개 팝업 제거
    time.sleep(2)
    print(len(driver.find_elements(By.LINK_TEXT, "상세보기")))
    
    if len(driver.find_elements(By.LINK_TEXT, "상세보기")) >= 15:
        #더보기 클릭 (클릭하지 않으면 첫페이지인 15개만 나온다)(2페이지로 이동하기와 같다)
        driver.find_element(By.XPATH, "//*[@id='info.search.place.more']").click()
        time.sleep(2)
    
    # 1~ 5페이지 링크 얻기
        page_urls = []
        for i in range(1,6):
            page = driver.find_element(By.ID, 'info.search.page.no' + str(i))
            page.click()
            time.sleep(2)
            urls = driver.find_elements(By.LINK_TEXT, "상세보기")
            for j in range(len(urls)):
                url = urls[j].get_attribute('href')
                print(url)
                page_urls.append(url)
    urls = driver.find_elements(By.LINK_TEXT, "상세보기")
    for j in range(len(urls)):
        url = urls[j].get_attribute('href')
        print(url)
        page_urls.append(url)

    columns = ['name','addresse','cat1','cat2','review_num','score_min', 'score', 'review','reviews_date','reviews_id']
    #사업장명, 주소, 음식종류1,음식종류2(메뉴),리뷰수,별점,리뷰
    df = pd.DataFrame(columns=columns)

    # driver = webdriver.Chrome(path)  # for Mac
    driver = webdriver.Chrome(executable_path=excutable_path)  # for Windows
    for page_url in page_urls:

        # 상세보기 페이지에 접속합니다
        driver.get(page_url)
        time.sleep(5)
        #사업장명
        name =  driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text
        # 사업장 주소
        address = driver.find_element(By.CLASS_NAME, "txt_address").text
        #평균 별점
        score_min = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text
        # 음식 종류
        cat1 =  driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/span[1]').text
        # 메뉴
        cat2 = []
        menus = driver.find_elements(By.CLASS_NAME, "info_menu")
        for menu in menus:
            cat2.append(menu.text)
        if driver.find_elements(By.CLASS_NAME, "box_grade_off"): #후기를 제공하지 않는 맛집 넘기기
            continue
        else:
            #리뷰수
            review_num = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[2]/span').text
            # 첫 페이지 리뷰를 크롤링합니다
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

            # 별점을 가져옵니다.
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})

            # 리뷰를 가져옵니다.
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
            
            # 리뷰를 쓴 날짜를 가져옵니다.
            reviews_dates = contents_div.find_all(name="span", attrs={"class":"time_write"})
            
            #리뷰 아이디 가져오기
            reviews_ids = contents_div.find_all(name="a", attrs={"class":"link_user"})
            
            #데이터프레임으로 정리합니다.
            for rate, review,reviews_date,reviews_id in zip(rates, reviews,reviews_dates,reviews_ids):
                row = [name,address,cat1,cat2,review_num,score_min,rate.text[0], review.find(name="span").text,reviews_date.text,reviews_id.text]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)


            # 2-5페이지의 리뷰를 크롤링합니다 과정은 위와 같습니다.
            for button_num in range(2, 6):
                # 오류가 나는 경우(리뷰 페이지가 없는 경우), 수행하지 않습니다.
                try:
                    another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
                    another_reviews.click()
                    time.sleep(2)

                    # 페이지 리뷰를 크롤링합니다
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

                    # 별점을 가져옵니다.
                    rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
                    # 리뷰를 가져옵니다.
                    reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
                    
                    # 리뷰를 쓴 날짜를 가져옵니다.
                    reviews_dates = contents_div.find_all(name="span", attrs={"class":"time_write"})
                    #리뷰 아이디 가져오기
                    reviews_ids = contents_div.find_all(name="a", attrs={"class":"link_user"})

                    #데이터프레임으로 정리합니다.
                    for rate, review,reviews_date,reviews_id in zip(rates, reviews,reviews_dates,reviews_ids):
                        row = [name,address,cat1,cat2,review_num,score_min,rate.text[0], review.find(name="span").text,reviews_date.text,reviews_id.text]
                        series = pd.Series(row, index=df.columns)
                        df = df.append(series, ignore_index=True)

                except:
                    break    
    driver.close()
    df.to_csv('{0}역.csv'.format(region), index=False) #index label을 따로 저장하지 않기

C:\Users\max_s\AppData\Local\Temp/ipykernel_57200/3595183025.py:113: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=excutable_path)


신당


C:\Users\max_s\AppData\Local\Temp/ipykernel_57200/3595183025.py:117: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
C:\Users\max_s\AppData\Local\Temp/ipykernel_57200/3595183025.py:121: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")


15
https://place.map.kakao.com/25036918
https://place.map.kakao.com/27584176
https://place.map.kakao.com/1285325677
https://place.map.kakao.com/1168606226
https://place.map.kakao.com/384441682
https://place.map.kakao.com/1012262230
https://place.map.kakao.com/1879590476
https://place.map.kakao.com/2129205762
https://place.map.kakao.com/16903726
https://place.map.kakao.com/7992623
https://place.map.kakao.com/252792288
https://place.map.kakao.com/1720827540
https://place.map.kakao.com/26993295
https://place.map.kakao.com/10439711
https://place.map.kakao.com/77215610
https://place.map.kakao.com/27388900
https://place.map.kakao.com/706318241
https://place.map.kakao.com/1282659967
https://place.map.kakao.com/19031971
https://place.map.kakao.com/13093208
https://place.map.kakao.com/21029575
https://place.map.kakao.com/1624572111
https://place.map.kakao.com/954591175
https://place.map.kakao.com/2093513784
https://place.map.kakao.com/907221168
https://place.map.kakao.com/19162981
https://place

C:\Users\max_s\AppData\Local\Temp/ipykernel_57200/3595183025.py:162: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=excutable_path)  # for Windows
C:\Users\max_s\AppData\Local\Temp/ipykernel_57200/3595183025.py:214: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")


...

https://place.map.kakao.com/8632941
https://place.map.kakao.com/2039198357
https://place.map.kakao.com/1272355946
https://place.map.kakao.com/644228765
https://place.map.kakao.com/21439806
https://place.map.kakao.com/27439556
https://place.map.kakao.com/17100643
https://place.map.kakao.com/7922550
https://place.map.kakao.com/17594595
https://place.map.kakao.com/8552779
https://place.map.kakao.com/1548153171
https://place.map.kakao.com/1472163077
https://place.map.kakao.com/18785524
https://place.map.kakao.com/25036918
https://place.map.kakao.com/8550495
https://place.map.kakao.com/1850802138
https://place.map.kakao.com/1443254669
https://place.map.kakao.com/18215030
https://place.map.kakao.com/19926635
https://place.map.kakao.com/723528091
https://place.map.kakao.com/239088283
https://place.map.kakao.com/1795239140
https://place.map.kakao.com/16818418
https://place.map.kakao.com/27321119
https://place.map.kakao.com/594655526
https://place.map.kakao.com/23666944
https://place.map.kakao.

화랑대
15
https://place.map.kakao.com/16414503
https://place.map.kakao.com/8083654
https://place.map.kakao.com/872257394
https://place.map.kakao.com/16999919
https://place.map.kakao.com/20553565
https://place.map.kakao.com/818423534
https://place.map.kakao.com/1137642618
https://place.map.kakao.com/1520080778
https://place.map.kakao.com/1117657731
https://place.map.kakao.com/1169910256
https://place.map.kakao.com/1653381358
https://place.map.kakao.com/27244691
https://place.map.kakao.com/18722630
https://place.map.kakao.com/20749857
https://place.map.kakao.com/153487369
https://place.map.kakao.com/27229512
https://place.map.kakao.com/2026803402
https://place.map.kakao.com/1826529353
https://place.map.kakao.com/1338966452
https://place.map.kakao.com/2013086833
https://place.map.kakao.com/20746565
https://place.map.kakao.com/26927344
https://place.map.kakao.com/206091656
https://place.map.kakao.com/1571140539
https://place.map.kakao.com/27478521
https://place.map.kakao.com/944088477
https:/

https://place.map.kakao.com/1840672910
https://place.map.kakao.com/1147452616
https://place.map.kakao.com/11658298
https://place.map.kakao.com/1771824067
https://place.map.kakao.com/26357129
https://place.map.kakao.com/22484529
https://place.map.kakao.com/1251846939
https://place.map.kakao.com/12765646
https://place.map.kakao.com/11123498
https://place.map.kakao.com/18347958
https://place.map.kakao.com/18376376
https://place.map.kakao.com/14608358
https://place.map.kakao.com/11401521
https://place.map.kakao.com/954423286
https://place.map.kakao.com/15658631
https://place.map.kakao.com/27549051
https://place.map.kakao.com/1768894016
https://place.map.kakao.com/18454137
https://place.map.kakao.com/2099464862
https://place.map.kakao.com/618020526
https://place.map.kakao.com/27510661
https://place.map.kakao.com/1151652295
https://place.map.kakao.com/1796473441
https://place.map.kakao.com/2125653901
https://place.map.kakao.com/26904863
https://place.map.kakao.com/617210652
https://place.map

방화
15
https://place.map.kakao.com/16989715
https://place.map.kakao.com/25858128
https://place.map.kakao.com/1943025765
https://place.map.kakao.com/9934681
https://place.map.kakao.com/15657241
https://place.map.kakao.com/27223284
https://place.map.kakao.com/482613052
https://place.map.kakao.com/16821857
https://place.map.kakao.com/8085593
https://place.map.kakao.com/16046189
https://place.map.kakao.com/18446644
https://place.map.kakao.com/1892570551
https://place.map.kakao.com/20884502
https://place.map.kakao.com/1096085576
https://place.map.kakao.com/1975000292
https://place.map.kakao.com/18489130
https://place.map.kakao.com/10057040
https://place.map.kakao.com/1246170867
https://place.map.kakao.com/16676118
https://place.map.kakao.com/9963657
https://place.map.kakao.com/18454650
https://place.map.kakao.com/489528513
https://place.map.kakao.com/2088745005
https://place.map.kakao.com/26644159
https://place.map.kakao.com/689944791
https://place.map.kakao.com/1738748547
https://place.map.

https://place.map.kakao.com/8476706
https://place.map.kakao.com/1912511834
https://place.map.kakao.com/914437083
https://place.map.kakao.com/21360025
https://place.map.kakao.com/27441113
https://place.map.kakao.com/14080398
https://place.map.kakao.com/505683747
https://place.map.kakao.com/218652450
https://place.map.kakao.com/1902068317
https://place.map.kakao.com/17009788
https://place.map.kakao.com/801687153
https://place.map.kakao.com/26354962
https://place.map.kakao.com/16042078
https://place.map.kakao.com/12526358
https://place.map.kakao.com/1198904049
https://place.map.kakao.com/1537507841
https://place.map.kakao.com/2040294187
https://place.map.kakao.com/1663159789
https://place.map.kakao.com/55847963
https://place.map.kakao.com/478938240
https://place.map.kakao.com/1113812585
https://place.map.kakao.com/10251373
https://place.map.kakao.com/2004780039
https://place.map.kakao.com/1142208147
https://place.map.kakao.com/11941074
https://place.map.kakao.com/194994590
https://place.m

## 데이터 확인하기

In [ ]:
df